In [ ]:
from geopy.distance import great_circle
import pandas as pd

In [ ]:
class CountNearThings:
  def __init__(self,df,lat,lon,dist=1):
    """
        df: 데이터 프레임
        lat: 중심 위도
        lon: 중심 경도
        dist: 기준 거리(km)"""
    self.df = df
    self.lat = lat
    self.lon = lon
    self.dist = dist

  def filter_by_rectangle(self):
      """
      사각 범위 내 데이터 필터링
      """
      lat_min = self.lat - 0.01 * self.dist
      lat_max = self.lat + 0.01 * self.dist

      lon_min = self.lon - 0.015 * self.dist
      lon_max = self.lon + 0.015 * self.dist

      self.points = [[lat_min, lon_min], [lat_max, lon_max]]

      result = self.df.loc[
          (self.df['lat'] > lat_min) &
          (self.df['lat'] < lat_max) &
          (self.df['lon'] > lon_min) &
          (self.df['lon'] < lon_max)
      ]
      result.index = range(len(result))

      return result

  def filter_by_radius(self, radius):
    """
    반경 범위 내 데이터 필터링
    """
  # 사각 범위 내 데이터 필터링
    tmp = self.filter_by_rectangle()

    # 기준 좌표 포인트
    center = (self.lat, self.lon)

    result = pd.DataFrame()
    for index, row in tmp.iterrows():
        # 개별 좌표 포인트
        point = (row['lat'], row['lon'])
        d = great_circle(center, point).kilometers
        if d <= radius:
            result = pd.concat([result, tmp.iloc[index, :].to_frame().T])

    result.index = range(len(result))

    return result

In [ ]:
df = pd.read_csv('tanghulu.csv', encoding='utf-8')
df = df.rename(columns={'탕후루/latitude': 'lat', '탕후루/longitude': 'lon'})

df
# df = df.drop_duplicates(['역명'])

,탕후루/market_name,탕후루/road_address,lat,lon,탕후루/utm-k.x,탕후루/utm-k.y
0,왕가탕후루 강남역점,서울 강남구 강남대로102길 14 장연빌딩 103호,37.501896,127.027041,958385.506108,1.944536e+06
1,왕가탕후루,서울 성북구 동소문로22길 36,37.591474,127.017991,957636.342562,1.954478e+06
2,왕가 탕후루,서울 종로구 수표로28길 17-30 (익선동) 왕가탕후루,37.573537,126.989595,955118.699275,1.952501e+06
3,달콤왕가탕후루 한양대점,서울 성동구 마조로 25 102호,37.560068,127.040943,959645.620175,1.950983e+06
4,왕가탕후루 잠실새내역점,서울 송파구 백제고분로7길 49 1층 107호,37.510949,127.084337,963454.215679,1.945516e+06
...,...,...,...,...,...,...
129,달콤왕가탕후루이대점,서울 서대문구 이화여대길 57,37.558926,126.945480,951213.529785,1.950902e+06
130,유니네탕후루,서울 관악구 난우10길 7 1층,37.476277,126.918174,948745.217641,1.941747e+06
131,탕탕탕후루,서울 노원구 한글비석로22길 30 1층,37.660855,127.073914,962608.284332,1.962152e+06
132,달콤왕가탕후루망원역점,서울 마포구 월드컵로13길 27,37.555439,126.908648,947957.991963,1.950535e+06


In [ ]:
cnt = CountNearThings(df,37.51429,127.0621,2) # 위에서 불러온 데이터 프레임 / 중심 좌표 / 중심 좌표 / 반경

result_radius = cnt.filter_by_radius(2)
print(result_radius)

  탕후루/market_name                      탕후루/road_address        lat  \
0    왕가탕후루 잠실새내역점             서울 송파구 백제고분로7길 49 1층 107호  37.510949   
1    달콤왕가탕후루 한티역점       서울 강남구 도곡로 405 삼환아르누보2, 1층 106호  37.497383   
2     달콤왕가탕후루 은마점                서울 강남구 삼성로 301 1층 104호   37.49908   
3    황후 탕후루 대치동본점  서울 강남구 역삼로 420 대치동 909 외 1필지 내제 1층4호  37.501001   
4    달콤왕가탕후루 선릉역점                서울 강남구 선릉로 424 지상1층 5호  37.502918   
5           판다탕후루         서울 강남구 도곡로 501 지하1층 B102(일부)호  37.499332   
6           청담탕후루             서울 강남구 도산대로85길 30 1층 117호  37.526438   

          lon    탕후루/utm-k.x     탕후루/utm-k.y  
0  127.084337  963454.215679  1945516.097864  
1  127.054724  960829.998886  1944022.959068  
2  127.060425  961334.821517  1944208.859234  
3  127.052907  960671.304475  1944425.125913  
4  127.050088  960423.066319  1944639.012737  
5  127.061138  961397.975761  1944236.612799  
6  127.049679  960399.445907  1947248.637499  


In [ ]:

# !! ______________________________이 부분을 도보로 걷는 거리 측정하는 코드로 변경하면 됩니다. !! ______________________________________________

# 설정한 중심 좌표와 각 역까지의 직선 거리 계산 함수

import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians

def mydistance(a1,b1,a2,b2):

  R = 6373.0

  x1 = radians(a1)
  y1 = radians(b1)
  x2 = radians(a2)
  y2 = radians(b2)

  dlon = x2 - x1
  dlat = y2 - y1

  a = sin(dlat / 2)**2 + cos(y1) * cos(y2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  return R * c


In [ ]:
# 측정 거리에 따른 weight 부여 (반경 2km 기준)
# ! 0.0 km ~ 0.4 km = 3.0
# ! 0.4 km ~ 0.8 km = 2.5
# ! 0.8 km ~ 1.2 km = 2.0
# ! 1.2 km ~ 1.6 km = 1.5
# ! 1.6 km ~ 2.0 km = 1.0

def weight_sum(distance_list):

  weight_list = []

  for k in distance_list:

    if k <= 0.4:
      weight_list.append(3.0)

    elif 0.4 <= k and k <= 0.8:
     weight_list.append(2.5)

    elif 0.8 <= k and k <= 1.2:
      weight_list.append(2.0)

    elif 1.2 <= k and k <= 1.6:
     weight_list.append(1.5)

    else:
     weight_list.append(1.0)

  return sum(weight_list)



### 즉, 중심 좌표 (37.51429, 127.0621)에서 '지하철역'에 대한 값은 10.0

In [ ]:
# 중심 좌표 -> 최종 웨이트값 한번에 계산하는 코드
''' _____________________________________________________________________ '''

# 예시 위도, 경도 데이터
lat = [37.51429, 37.562832, 37.638951]
lon = [127.0621, 127.007815, 127.024745]

station = [] # 중심 좌표별 지하철역에 대한 최종 웨이트값 리스트

for i in range(len(lat)):

  cnt = CountNearThings(df,lat[i], lon[i], 2)

  result_radius = cnt.filter_by_radius(2)

  distance_list = []

  for j in range(len(result_radius)):
    distance_list.append(mydistance(lat[i],lon[i], result_radius.iloc[j][result_radius.columns.get_loc('lat')], result_radius.iloc[j][result_radius.columns.get_loc('lon')]))

  station.append(weight_sum(distance_list))

print(station)